# Benchmark Linear Image Classification on ImageNet-1K

In this tutorial, we look at a simple example of how to use VISSL to run linear image classification benchmark for [ResNet-50 Torchvision pre-trained model](https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py#L16). This benchmark freezes the model trunk and attaches a linear MLP on top of the trunk features.

You can make a copy of this tutorial by `File -> Open in playground mode` and make changes there. DO NOT request access to this tutorial.

**NOTE:** Please ensure your Collab Notebook has GPU available. To ensure/select this, simple follow: `Edit -> Notebook Settings -> select GPU`.

## Install VISSL

Installing VISSL is pretty straightfoward. We will use pip binaries of VISSL and follow instructions from [here](https://github.com/facebookresearch/vissl/blob/master/INSTALL.md#install-vissl-pip-package).

In [ ]:
# Install: PyTorch (we assume 1.5.1 but VISSL works with all PyTorch versions >=1.4)
!pip install torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

# install opencv
!pip install opencv-python

# install apex by checking system settings: cuda version, pytorch version, python version
import sys
import torch
version_str="".join([
    f"py3{sys.version_info.minor}_cu",
    torch.version.cuda.replace(".",""),
    f"_pyt{torch.__version__[0:5:2]}"
])
print(version_str)

# install apex (pre-compiled with optimizer C++ extensions and CUDA kernels)
!pip install apex -f https://dl.fbaipublicfiles.com/vissl/packaging/apexwheels/{version_str}/download.html

# install VISSL
!pip install vissl -f https://dl.fbaipublicfiles.com/vissl/packaging/visslwheels/download.html

VISSL should be successfuly installed by now and all the dependencies should be available.

In [ ]:
import vissl
import tensorboard
import apex
import torch

## YAML config file for Linear benchmark

VISSL provides yaml configuration files for all benchmark tasks including linear image classification on ImageNet [here](https://github.com/facebookresearch/vissl/tree/master/configs/config/benchmark). 

For the purpose of this tutorial, we will use [this config file](https://github.com/facebookresearch/vissl/blob/master/configs/config/test/integration_test/quick_eval_in1k_linear_imagefolder_head.yaml) for training a linear classifier on the trunk output of ResNet-50 supervised model on 1-gpu. Let's go ahead and download the [example config file](https://github.com/facebookresearch/vissl/blob/master/configs/config/test/integration_test/quick_eval_in1k_linear_imagefolder_head.yaml).


In [2]:
!mkdir -p configs/config
!wget -q -O configs/__init__.py https://dl.fbaipublicfiles.com/vissl/tutorials/configs/__init__.py 
!wget -q -O configs/config/eval_in1k_linear_imagefolder_head.yaml https://dl.fbaipublicfiles.com/vissl/tutorials/configs/eval_in1k_linear_imagefolder_head.yaml

## Download the ResNet-50 weights from Torchvision

We download the weights from the [torchvision ResNet50 model](https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py#L16):

In [ ]:
!wget https://download.pytorch.org/models/resnet50-19c8e357.pth

## Builtin training tool in VISSL

VISSL also provides a [helper python tool](https://github.com/facebookresearch/vissl/blob/master/tools/run_distributed_engines.py) that allows to use VISSL for training purposes. This tool offers:
- allows training and feature extraction both using VISSL. 
- also allows training on 1-gpu or multi-gpu. 
- can be used to launch multi-machine distributed training.

Let's go ahead and download this tool directly.

In [ ]:
!wget https://dl.fbaipublicfiles.com/vissl/tutorials/run_distributed_engines.py

## Creating a custom data

For the purpose of this tutorial, since we don't have ImageNet on the disk, we will create a dummy dataset by copying an image from COCO dataset in ImageNet dataset folder style as below:

In [5]:
!mkdir -p dummy_data/train/class1
!mkdir -p dummy_data/train/class2
!mkdir -p dummy_data/val/class1
!mkdir -p dummy_data/val/class2

# create 2 classes in train and add 2 images per class
!wget http://images.cocodataset.org/val2017/000000439715.jpg -q -O dummy_data/train/class1/img1.jpg
!wget http://images.cocodataset.org/val2017/000000439715.jpg -q -O dummy_data/train/class1/img2.jpg
!wget http://images.cocodataset.org/val2017/000000439715.jpg -q -O dummy_data/train/class1/img3.jpg
!wget http://images.cocodataset.org/val2017/000000439715.jpg -q -O dummy_data/train/class1/img4.jpg
!wget http://images.cocodataset.org/val2017/000000439715.jpg -q -O dummy_data/train/class1/img5.jpg

!wget http://images.cocodataset.org/val2017/000000439715.jpg -q -O dummy_data/train/class2/img1.jpg
!wget http://images.cocodataset.org/val2017/000000439715.jpg -q -O dummy_data/train/class2/img2.jpg
!wget http://images.cocodataset.org/val2017/000000439715.jpg -q -O dummy_data/train/class2/img3.jpg
!wget http://images.cocodataset.org/val2017/000000439715.jpg -q -O dummy_data/train/class2/img4.jpg
!wget http://images.cocodataset.org/val2017/000000439715.jpg -q -O dummy_data/train/class2/img5.jpg

# create 2 classes in val and add 2 images per class
!wget http://images.cocodataset.org/val2017/000000439715.jpg -q -O dummy_data/val/class1/img1.jpg
!wget http://images.cocodataset.org/val2017/000000439715.jpg -q -O dummy_data/val/class1/img2.jpg
!wget http://images.cocodataset.org/val2017/000000439715.jpg -q -O dummy_data/val/class1/img3.jpg
!wget http://images.cocodataset.org/val2017/000000439715.jpg -q -O dummy_data/val/class1/img4.jpg
!wget http://images.cocodataset.org/val2017/000000439715.jpg -q -O dummy_data/val/class1/img5.jpg

!wget http://images.cocodataset.org/val2017/000000439715.jpg -q -O dummy_data/val/class2/img1.jpg
!wget http://images.cocodataset.org/val2017/000000439715.jpg -q -O dummy_data/val/class2/img2.jpg
!wget http://images.cocodataset.org/val2017/000000439715.jpg -q -O dummy_data/val/class2/img3.jpg
!wget http://images.cocodataset.org/val2017/000000439715.jpg -q -O dummy_data/val/class2/img4.jpg
!wget http://images.cocodataset.org/val2017/000000439715.jpg -q -O dummy_data/val/class2/img5.jpg


## Using the custom data in VISSL

Next step for us is to register the dummy data we created above with VISSL. Registering the dataset involves telling VISSL about the dataset name and the paths for the dataset. For this, we create a simple json file with the metadata and save it to `configs/config/dataset_catalog.py` file.

**NOTE**: VISSL uses the specific `dataset_catalog.json` under the path `configs/config/dataset_catalog.json`.

In [6]:
json_data = {
    "dummy_data_folder": {
      "train": [
        "/content/dummy_data/train", "/content/dummy_data/train"
      ],
      "val": [
        "/content/dummy_data/val", "/content/dummy_data/val"
      ]
    }
}

# use VISSL's api to save or you can use your custom code.
from vissl.utils.io import save_file
save_file(json_data, "/content/configs/config/dataset_catalog.json")

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 



Next, we verify that the dataset is registered with VISSL. For that we query VISSL's dataset catalog as below:

In [7]:
from vissl.data.dataset_catalog import VisslDatasetCatalog

# list all the datasets that exist in catalog
print(VisslDatasetCatalog.list())

# get the metadata of dummy_data_folder dataset
print(VisslDatasetCatalog.get("dummy_data_folder"))

['dummy_data_folder']
{'train': ['/content/dummy_data/train', '/content/dummy_data/train'], 'val': ['/content/dummy_data/val', '/content/dummy_data/val']}


## Training linear classifier on trunk output

We are ready to train the linear classifier now. For the purpose of this tutorial, we will use synthetic dataset and train on dummy images. VISSL supports training on wide range of datasets and allows adding custom datasets. Please see VISSL documentation on how to use the datasets. To train on ImageNet instead: assuming your ImageNet dataset folder path is `/path/to/my/imagenet/folder/`, you can add the following command line 
input to your training command: 
```
config.DATA.TRAIN.DATASET_NAMES=[imagenet1k_folder] \
config.DATA.TRAIN.DATA_SOURCES=[disk_folder] \
config.DATA.TRAIN.DATA_PATHS=["/path/to/my/imagenet/folder/train"] \
config.DATA.TRAIN.LABEL_SOURCES=[disk_folder]
```

The training command looks like:

In [9]:
!python3 run_distributed_engines.py \
    hydra.verbose=true \
    config=eval_in1k_linear_imagefolder_head \
    config.DATA.TRAIN.DATA_SOURCES=[disk_folder] \
    config.DATA.TRAIN.LABEL_SOURCES=[disk_folder] \
    config.DATA.TRAIN.DATASET_NAMES=[dummy_data_folder] \
    config.DATA.TRAIN.BATCHSIZE_PER_REPLICA=2 \
    config.DATA.TEST.DATA_SOURCES=[disk_folder] \
    config.DATA.TEST.LABEL_SOURCES=[disk_folder] \
    config.DATA.TEST.DATASET_NAMES=[dummy_data_folder] \
    config.DATA.TEST.BATCHSIZE_PER_REPLICA=2 \
    config.DISTRIBUTED.NUM_NODES=1 \
    config.DISTRIBUTED.NUM_PROC_PER_NODE=1 \
    config.CHECKPOINT.DIR="./checkpoints" \
    config.MODEL.WEIGHTS_INIT.PARAMS_FILE="/content/resnet50-19c8e357.pth" \
    config.MODEL.WEIGHTS_INIT.APPEND_PREFIX="trunk._feature_blocks." \
    config.MODEL.WEIGHTS_INIT.STATE_DICT_KEY_NAME=""


** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 

####### overrides: ['hydra.verbose=true', 'config=eval_in1k_linear_imagefolder_head', 'config.DATA.TRAIN.DATA_SOURCES=[disk_folder]', 'config.DATA.TRAIN.LABEL_SOURCES=[disk_folder]', 'config.DATA.TRAIN.DATASET_NAMES=[dummy_data_folder]', 'config.DATA.TRAIN.BATCHSIZE_PER_REPLICA=2', 'config.DATA.TEST.DATA_SOURCES=[disk_folder]', 'config.DATA.TEST.LABEL_SOURCES=[disk_folder]', 'config.DATA.TEST.DATASET_NAMES=[dummy_data_folder]', 'config.DATA.TEST.BATCHSIZE_PER_REPLICA=2', 'config.DISTRIBUTED.NUM_NODES=1', 'config.DISTRIBUTED.NUM_PROC_PER_NODE=1', 'config.CHECKPOINT.DIR=./checkpoints', 'config.MODEL.WEIGHTS_INIT.PARAMS_FILE=/content/resnet50-19c8e357.pth', 'config.MODEL.WEIGHTS_INIT.APPEND_PREFIX=trunk._feature_blocks.', 'config.MODEL.WEIGHTS_INIT.STATE_DICT_KEY_NAME=', 'hydra.verbose=true']
INFO 2021-01-13 21:58:46,864 __init__.py:

And we are done!! We have the linear classifier trained on the trunk output and the `metrics.json` containing `top-1` and `top-5` accuracy on validation set is available in `checkpoints/metrics.json`.

In [10]:
ls checkpoints/

log.txt  metrics.json  model_final_checkpoint_phase2.torch  model_phase0.torch


In [16]:
cat checkpoints/metrics.json

{"iteration": 5, "phase_idx": 0, "train_accuracy_list_meter": {"top_1": {"0": 20.0}, "top_5": {"0": 20.0}}, "train_phase_idx": 0}
{"iteration": 5, "phase_idx": 1, "test_accuracy_list_meter": {"top_1": {"0": 50.0}, "top_5": {"0": 50.0}}, "train_phase_idx": 0}
{"iteration": 10, "phase_idx": 2, "train_accuracy_list_meter": {"top_1": {"0": 40.0}, "top_5": {"0": 40.0}}, "train_phase_idx": 1}
{"iteration": 10, "phase_idx": 3, "test_accuracy_list_meter": {"top_1": {"0": 50.0}, "top_5": {"0": 50.0}}, "train_phase_idx": 1}


## Training linear classifiers on several trunk features

VISSL also supports training linear classifiers on several features of the trunk. For the purpose of tutorial, we will use [this](https://github.com/facebookresearch/vissl/blob/master/configs/config/test/integration_test/quick_eval_in1k_linear_imagefolder.yaml) config file. Let's go ahead and download it.

In [11]:
!wget -q -O configs/config/eval_in1k_linear_imagefolder.yaml https://dl.fbaipublicfiles.com/vissl/tutorials/configs/eval_in1k_linear_imagefolder.yaml

Now, let's re-run the previous command:

In [13]:
!python3 run_distributed_engines.py \
    hydra.verbose=true \
    config=eval_in1k_linear_imagefolder \
    config.DATA.TRAIN.DATA_SOURCES=[disk_folder] \
    config.DATA.TRAIN.LABEL_SOURCES=[disk_folder] \
    config.DATA.TRAIN.DATASET_NAMES=[dummy_data_folder] \
    config.DATA.TRAIN.BATCHSIZE_PER_REPLICA=2 \
    config.DATA.TEST.DATA_SOURCES=[disk_folder] \
    config.DATA.TEST.LABEL_SOURCES=[disk_folder] \
    config.DATA.TEST.DATASET_NAMES=[dummy_data_folder] \
    config.DATA.TEST.BATCHSIZE_PER_REPLICA=2 \
    config.DISTRIBUTED.NUM_NODES=1 \
    config.DISTRIBUTED.NUM_PROC_PER_NODE=1 \
    config.CHECKPOINT.DIR="./checkpoints_trunk_eval" \
    config.MODEL.WEIGHTS_INIT.PARAMS_FILE="/content/resnet50-19c8e357.pth" \
    config.MODEL.WEIGHTS_INIT.APPEND_PREFIX="trunk.base_model._feature_blocks." \
    config.MODEL.WEIGHTS_INIT.STATE_DICT_KEY_NAME=""

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 

####### overrides: ['hydra.verbose=true', 'config=eval_in1k_linear_imagefolder', 'config.DATA.TRAIN.DATA_SOURCES=[disk_folder]', 'config.DATA.TRAIN.LABEL_SOURCES=[disk_folder]', 'config.DATA.TRAIN.DATASET_NAMES=[dummy_data_folder]', 'config.DATA.TRAIN.BATCHSIZE_PER_REPLICA=2', 'config.DATA.TEST.DATA_SOURCES=[disk_folder]', 'config.DATA.TEST.LABEL_SOURCES=[disk_folder]', 'config.DATA.TEST.DATASET_NAMES=[dummy_data_folder]', 'config.DATA.TEST.BATCHSIZE_PER_REPLICA=2', 'config.DISTRIBUTED.NUM_NODES=1', 'config.DISTRIBUTED.NUM_PROC_PER_NODE=1', 'config.CHECKPOINT.DIR=./checkpoints_trunk_eval', 'config.MODEL.WEIGHTS_INIT.PARAMS_FILE=/content/resnet50-19c8e357.pth', 'config.MODEL.WEIGHTS_INIT.APPEND_PREFIX=trunk.base_model._feature_blocks.', 'config.MODEL.WEIGHTS_INIT.STATE_DICT_KEY_NAME=', 'hydra.verbose=true']
INFO 2021-01-13 22:05:08

And we are done!! We have the linear classifier trained on the trunk features `res5` and `res5avg` and the `metrics.json` containing `top-1` and `top-5` accuracy on validation set is available in `checkpoints_trunk_eval/metrics.json`.

In [14]:
ls checkpoints_trunk_eval/

log.txt  metrics.json  model_final_checkpoint_phase2.torch  model_phase0.torch


In [15]:
cat checkpoints_trunk_eval/metrics.json

{"iteration": 5, "phase_idx": 0, "train_accuracy_list_meter": {"top_1": {"res5": 0.0, "res5avg": 0.0}, "top_5": {"res5": 40.0, "res5avg": 0.0}}, "train_phase_idx": 0}
{"iteration": 5, "phase_idx": 1, "test_accuracy_list_meter": {"top_1": {"res5": 50.0, "res5avg": 50.0}, "top_5": {"res5": 50.0, "res5avg": 50.0}}, "train_phase_idx": 0}
{"iteration": 10, "phase_idx": 2, "train_accuracy_list_meter": {"top_1": {"res5": 60.0, "res5avg": 10.0}, "top_5": {"res5": 80.0, "res5avg": 30.0}}, "train_phase_idx": 1}
{"iteration": 10, "phase_idx": 3, "test_accuracy_list_meter": {"top_1": {"res5": 50.0, "res5avg": 0.0}, "top_5": {"res5": 100.0, "res5avg": 50.0}}, "train_phase_idx": 1}


# Loading Pre-trained models in VISSL

VISSL supports Torchvision models out of the box. Generally, for loading any non-VISSL model, one needs to correctly set the following configuration options:

```yaml
WEIGHTS_INIT:
  # path to the .torch weights files
  PARAMS_FILE: ""
  # name of the state dict. checkpoint = {"classy_state_dict": {layername:value}}. Options:
  #   1. classy_state_dict - if model is trained and checkpointed with VISSL.
  #      checkpoint = {"classy_state_dict": {layername:value}}
  #   2. "" - if the model_file is not a nested dictionary for model weights i.e.
  #      checkpoint = {layername:value}
  #   3. key name that your model checkpoint uses for state_dict key name.
  #      checkpoint = {"your_key_name": {layername:value}}
  STATE_DICT_KEY_NAME: "classy_state_dict"
  # specify what layer should not be loaded. Layer names with this key are not copied
  # By default, set to BatchNorm stats "num_batches_tracked" to be skipped.
  SKIP_LAYERS: ["num_batches_tracked"]
  ####### If loading a non-VISSL trained model, set the following two args carefully #########
  # to make the checkpoint compatible with VISSL, if you need to remove some names
  # from the checkpoint keys, specify the name
  REMOVE_PREFIX: ""
  # In order to load the model (if not trained with VISSL) with VISSL, there are 2 scenarios:
  #    1. If you are interested in evaluating the model features and freeze the trunk.
  #       Set APPEND_PREFIX="trunk.base_model." This assumes that your model is compatible
  #       with the VISSL trunks. The VISSL trunks start with "_feature_blocks." prefix. If
  #       your model doesn't have these prefix you can append them. For example:
  #       For TorchVision ResNet trunk, set APPEND_PREFIX="trunk.base_model._feature_blocks."
  #    2. where you want to load the model simply and finetune the full model.
  #       Set APPEND_PREFIX="trunk."
  #       This assumes that your model is compatible with the VISSL trunks. The VISSL
  #       trunks start with "_feature_blocks." prefix. If your model doesn't have these
  #       prefix you can append them.
  #       For TorchVision ResNet trunk, set APPEND_PREFIX="trunk._feature_blocks."
  # NOTE: the prefix is appended to all the layers in the model
  APPEND_PREFIX: ""
  ```